---
## PEP 01

If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23.
Find the sum of all the multiples of 3 or 5 below 1000.

### Pure Python

In [ ]:
sum(range(3,1000,3)) + sum(range(5,1000,5)) - sum(range(15,1000,15))

In [ ]:
%%timeit
sum(range(3,10**4,3)) + sum(range(5,10**4,5)) - sum(range(15,10**4,15))

In [ ]:
sum(range(3,10**4,3)) + sum(range(5,10**4,5)) - sum(range(15,10**4,15))

### Numpy Version

In [ ]:
import numpy as np
n = np.arange(0,10**3)

In [ ]:
%%timeit
n[3:1000:3].sum() + n[5:1000:5].sum() - n[15:1000:15].sum()

In [ ]:
n = np.arange(0,10**4)

In [ ]:
%%timeit
n[3:10**6:3].sum() + n[5:10**6:5].sum() - n[15:10**6:15].sum()

In [ ]:
n[3:-1:3].sum() + n[5:-1:5].sum() - n[15:-1:15].sum()

In [ ]:
n[3:-1:3].sum() + n[5:-1:5].sum() - n[15:-1:15].sum()

***

# PEP 02
Each new term in the Fibonacci sequence is generated by adding the previous two terms. By starting with 1 and 2, the first 10 terms will be:

1, 2, 3, 5, 8, 13, 21, 34, 55, 89, ...

By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.

### Numpy Version

In [ ]:
fibs = np.int32(np.zeros(99)).reshape(33,3)  # dtype assignment

In [ ]:
fibs[0] = np.array([0,0,1])

In [ ]:
fibs[0]

In [ ]:
for i in range(1,33):
    tail  = fibs[i-1][1::].sum()
    fibs[i] = np.array([fibs[i-1][1],fibs[i-1][2],tail])
print(fibs)

In [ ]:
even_sum = 0
for item in fibs[::,-1]:
    if np.mod(item,2)==0:
        even_sum += item
print(even_sum)
    

***
##  PEP003
The prime factors of 13195 are 5, 7, 13 and 29.

What is the largest prime factor of the number 600851475143 ?

### Pure Python

In [ ]:
from math import sqrt
def factorize(n):
    res = []
    # iterate over all even numbers first.
    while n % 2 == 0:
        res.append(2)
        n //= 2
    # try odd numbers up to sqrt(n)
    limit = sqrt(n+1)
    i = 3
    while i <= limit:
        if n % i == 0:
            res.append(i)
            n //= i
            limit = sqrt(n+i)
        else:
            i += 2
    if n != 1:
        res.append(n)
    return res

In [ ]:
factorize(600851475143)

### Numpy Version

In [ ]:
np.sqrt(600851475143)

In [ ]:
numbers = np.arange(2,10**6);numbers

In [ ]:
mask = np.mod(600851475143,numbers)==0
mask

In [ ]:
def prime_seive(n): ##pure python -by myself
    seive = [False] *2 + [True]*(n-2)
    for i in range(2,n):
        if seive[i]:
            seive[i*i::i] = [False]*len(seive[i*i::i])
    return [k for k in range(n) if seive[k] ]

In [ ]:
def primes(n):
    """ Returns  a list of primes < n """
    sieve = [True] * n
    for i in range(3,int(n**0.5)+1,2):
        if sieve[i]:
            sieve[i*i::2*i]=[False]*((n-i*i-1)//(2*i)+1)
    return [2] + [i for i in range(3,n,2) if sieve[i]]

In [ ]:
for k in (numbers*mask):
    if k and k in primes(10**6):
        print(k)

In [ ]:
def primes_np(n):
    """ Returns a array of primes, 3 <= p < n """
    sieve = np.ones(n//2, dtype=np.bool)
    for i in range(3,int(n**0.5)+1,2):
        if sieve[i//2]:
            sieve[i*i//2::i] = False
    return 2*np.nonzero(sieve)[0][1::]+1

In [ ]:
def sieveOfEratosthenes(n):
    """sieveOfEratosthenes(n): return the list of the primes < n."""
    # Code from: <dickinsm@gmail.com>, Nov 30 2006
    # http://groups.google.com/group/comp.lang.python/msg/f1f10ced88c68c2d
    if n <= 2:
        return []
    sieve = range(3, n, 2)
    top = len(sieve)
    for si in sieve:
        if si:
            bottom = (si*si - 3) // 2
            if bottom >= top:
                break
            sieve[bottom::si] = [0] * -((bottom - top) // si)
    return [2] + [el for el in sieve if el]

In [ ]:
mask2 = np.mod(600851475143,primes_np(780000))==0
result = []
for factor in primes_np(780000)*mask2:
    if factor:
        result.append(factor)
result

In [ ]:
primes_np(780000)[mask2]

---
## PEP004
A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is 9009 = 91 × 99.
Find the largest palindrome made from the product of two 3-digit numbers.

### Pure Python

In [ ]:
%load_ext Cython

In [ ]:
%%cython
def isPalindrome(num):
    return str(num) == str(num)[::-1]
def largest(bot, top):
    z = 0
    for x in range(top, bot, -1):
        for y in range(top,bot, -1):
            if isPalindrome(x*y):
                if x*y > z:
                    z = x*y
    return z

In [ ]:
%%timeit 
print(largest(100,999))

## using cython speed 2x

### Numpy Version

In [ ]:
import numpy as np

In [ ]:
def isPalindrome(num):
    return str(num) == str(num)[::-1]    
check_p = np.frompyfunc(isPalindrome,1,1)

In [ ]:
%%timeit
arr = np.arange(100,999)
mesh = np.meshgrid(arr,arr)
products = mesh[0]*mesh[1]
mask01 = check_p(products)
mask02 = np.nonzero(mask01)  ## key!!!!
products[mask02].max()

In [ ]:
arr = np.arange(100,999)
mesh = np.meshgrid(arr,arr)
products = mesh[0]*mesh[1]
mask01 = check_p(products)
mask02 = np.nonzero(mask01)  ## key!!!!
products[mask02].max()

---
## PEP005
2520 is the smallest number that can be divided by each of the numbers from 1 to 10 without any remainder.

What is the smallest positive number that is evenly divisible by all of the numbers from 1 to 20?

### Pure Python

In [ ]:
p = prime_seive(20);p

In [ ]:
def max_pow(limit,k):
    i = 0
    while k**i<limit:
        i+=1
    return i-1

In [ ]:
power =[max_pow(20,i) for i in p];power

In [ ]:
t = zip(p,power);t

In [ ]:
gcd20 = 1
for k in t:
    gcd20 *= k[0]**k[1]
    print(k)
print(gcd20)
    

In [ ]:
def gcd(a, b):
    """
    @param a:
    @param b:
    @return:greatest common divisor
    """
    if a > b:
        a, b = b, a
    while True:
        mod = b % a
        if mod == 0:
            return a
        b, a = a, mod
def lcm(a,b):
    """
    @param a:
    @param b:
    @return: lease common multiple
    """
    return int(a*b/(gcd(a,b)))

In [ ]:
from functools import reduce
reduce(lcm,range(1,21))

### Numpy Version

In [ ]:
primes_np(21)

In [ ]:
p = np.array([2, 3,  5,  7, 11, 13, 17, 19])

In [ ]:
def max_pow(k):
    limit =20
    i = 0
    while k**i<limit:
        i+=1
    return i-1

In [ ]:
maxp = np.frompyfunc(max_pow,1,1)

In [ ]:
maxp(p)

In [ ]:
p**maxp(p)

In [ ]:
np.multiply.reduce(p**maxp(p))